In [94]:
import pyroomacoustics as pra
import soundfile as sf
import numpy as np
import pydub
import tensorflow as tf
import random
import IPython.display as ipd

In [95]:
sr=16000

In [96]:
tfrecord_file = '../_positive_00_0000.tfrecord'

In [97]:
examples = []
limit = 100
for i, example in enumerate(tf.python_io.tf_record_iterator(tfrecord_file)):
    eg_np = tf.train.Example.FromString(example)
    audio_segment = pydub.AudioSegment(
        eg_np.features.feature["audio"].bytes_list.value[0], 
        frame_rate=16000,
        sample_width=2, 
        channels=1
    )
    y = audio_segment.get_array_of_samples()
    examples.append(y)
    if i > limit:
        break

In [98]:
ex = examples[random.randint(0, len(examples) - 1)]

In [99]:
def get_room_opts():

    room_dim_opts = np.arange(10, 30, 1).tolist()
    room_absorption_opts = np.arange(0.05, .2, 0.05)

    room_l = room_dim_opts[random.randint(0, len(room_dim_opts) - 1)]
    room_w = room_dim_opts[random.randint(0, len(room_dim_opts) - 1)]
    room_h = room_dim_opts[random.randint(0, len(room_dim_opts) - 1)]
    room_dim = [room_l, room_w, room_h]

    source_x_pos = random.randint(0, 2*room_l) / 2
    source_y_pos = random.randint(0, 2*room_w) / 2
    source_z_pos = random.randint(0, 2*room_h) / 2
    source_pos = [source_x_pos, source_y_pos, source_z_pos]

    mic_x_pos = random.randint(0, 2*room_l) / 2
    mic_y_pos = random.randint(0, 2*room_w) / 2
    mic_z_pos = random.randint(0, 2*room_h) / 2
    mic_pos = np.c_[[mic_x_pos, mic_y_pos, mic_z_pos]]

    room_absorption = room_absorption_opts[random.randint(0, len(room_absorption_opts) - 1)]
    
    return room_dim, source_pos, mic_pos, room_absorption

In [100]:
def audio_augment_noisy(signal, room_dim, source_pos, mic_pos, room_absorption):
    room = pra.ShoeBox(room_dim, fs=16000, absorption=room_absorption)
    room.add_source(source_pos, signal=signal, delay=delay)
    mic_array = pra.MicrophoneArray(mic_pos, room.fs)
    room.add_microphone_array(mic_array)
    room.simulate()
    
    return room.mic_array.signals[0,:]

In [101]:
noisy_ex = []
for i in range(3):
    room_dim, source_pos, mic_pos, room_absorption = get_room_opts()
    noisy_ex.append(audio_augment_noisy(ex, room_dim, source_pos, mic_pos, room_absorption))

In [102]:
print("Original Example")
ipd.Audio(ex, rate=sr)

Original Example


In [103]:
print("Noisy Example 1")
ipd.Audio(noisy_ex[0], rate=sr)

Noisy Example 1


In [104]:
print("Noisy Example 2")
ipd.Audio(noisy_ex[1], rate=sr)

Noisy Example 2


In [105]:
print("Noisy Example 3")
ipd.Audio(noisy_ex[2], rate=sr)

Noisy Example 3
